# Task 1: Does increasing the bitrate or changing the game (independet variables) have a significant effect on the video quality (VQ) ratings (dependent variable)? Please consider only ratings at a resolution of 1080p and a framerate of 60 fps (conditions 36 & 50). Use the ratings provided in the gaming video quality data set.

### Step 1: Import libraries and read in data set

In [1]:
# install.packages('dplyr')      # processing 
# install.packages('gdata')      # file reading
# install.packages('car')        # homogenity of variances
# install.packages('rstatix')    # Tukey's post hoc test
# install.packages('ez')         # ANOVA table

In [2]:
library(dplyr)     # processing
library(readxl)    # reading in data
library(car)       # homogenity of variances
library(rstatix)   # Tukey's post hoc test
library(ez)        # ANOVA table

Warning message:
"package 'dplyr' was built under R version 3.6.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'car' was built under R version 3.6.2"Loading required package: carData

Attaching package: 'car'

The following object is masked from 'package:dplyr':

    recode

Warning message:
"package 'rstatix' was built under R version 3.6.2"
Attaching package: 'rstatix'

The following object is masked from 'package:stats':

    filter

Warning message:
"package 'ez' was built under R version 3.6.2"Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 
Warning message:
"package 'omics' was built under R version 3.6.2"

In [3]:
# read in data sets
get_gaming_data <- function() {
    gaming_data <- read_excel("datasets/DB01_gaming_video_quality_dataset.xlsx")
    gaming_data <- gaming_data %>% dplyr::filter(Resolution == 1080, 
                                      Framerate == 60, 
                                      (Condition == 36 | Condition == 50)) %>%
                                    select(PID, Bitrate, Game, VQ) %>%
                                    mutate(Bitrate = as.character(Bitrate), PID = as.factor(PID), Game = as.factor(Game)) %>%
                                    arrange(PID)
    
    gaming_data
}

gaming_data <- get_gaming_data()
head(gaming_data)

PID,Bitrate,Game,VQ
1,2000,Game1,2.9
1,2000,Game6,2.0
1,6000,Game1,4.5
1,6000,Game6,2.6
2,2000,Game1,4.6
2,2000,Game6,2.6


### Step 2: Decide on which ANOVA test to use

#### => 2 independent input variables (bitrate & game), 1 dependent variable, repeated measures for same subjects => two-way repeated measure ANOVA

### Step 3: Check assumptions

#### 1. Dependent variables on interval or ratio scale => check, because VQ is continuous
#### 2. Independent variables with two or more groups => check, because Game1 & Game6 for game, 2000 & 6000 kbps for bitrate
#### 3. Indepenence of observation => check, because participants are independent and within-subject independence is assumed
#### 4. No significant outliers => don't know, need to check that in the next step!
#### 5. Normally distributed population for every single group => don't know, need to check that in the next step!
#### 6. Homogenity of variances => don't know, need to check that in the next step!

### Step 3.1: Outlier detection

In [4]:
# z score method
head(
    gaming_data %>% 
                mutate(Std_Dev_VQ = sd(VQ), 
                       Mean_VQ = mean(VQ)) %>%
                mutate(Z_Score_VQ = (VQ - Mean_VQ) / Std_Dev_VQ) %>%
                select(VQ, Z_Score_VQ) %>%
                drop_na() %>%
                arrange(desc(Z_Score_VQ))
)

VQ,Z_Score_VQ
5.7,2.002825
5.7,2.002825
5.7,2.002825
5.6,1.906673
5.6,1.906673
5.2,1.522065


#### Criterion checked: no significant outliers (no absolute z score greater than 3.29)

### Step 3.2: Normally distributed population for every single group

In [5]:
# normality checking for groups
check_normality_for_group <- function(game, bitrate) {
    data <- gaming_data %>% dplyr::filter(Game == game & Bitrate == bitrate) 
    test_result <- ks.test(data[['VQ']], "pnorm", mean=mean(data[['VQ']]), sd=sd(data[['VQ']]))
    result_string <- paste0('Normality for ', game, ' & bitrate of ', bitrate, ' kbps: ')
    test_result[['p.value']]
    
    if(test_result[['p.value']] < 0.05) {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=3)), 
                                ' p-value (Kolmogrov-Smirnov) == NO, ')
    } else {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=3)), 
                                ' p-value (Kolmogrov-Smirnov) == YES, ')
    }
    
    test_result <- shapiro.test(data[['VQ']])
    if(test_result[['p.value']] < 0.05) {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=3)), 
                                ' p-value (Shapiro-Wilk) == NO!')
    } else {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=3)), 
                                ' p-value (Kolmogrov-Smirnov) == YES!')
    }
    
    result_string
}

In [6]:
# print normality tests
games <- (gaming_data %>% distinct(Game))[['Game']]
bitrates <- (gaming_data %>% distinct(Bitrate))[['Bitrate']]

for (game in games) {
    for (bitrate in bitrates) {
        print(check_normality_for_group(game, bitrate))
    }  
}

Warning message in ks.test(data[["VQ"]], "pnorm", mean = mean(data[["VQ"]]), sd = sd(data[["VQ"]])):
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for Game1 & bitrate of 2000 kbps: 0.249 p-value (Kolmogrov-Smirnov) == YES, 0.04 p-value (Shapiro-Wilk) == NO!"


Warning message in ks.test(data[["VQ"]], "pnorm", mean = mean(data[["VQ"]]), sd = sd(data[["VQ"]])):
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for Game1 & bitrate of 6000 kbps: 0.975 p-value (Kolmogrov-Smirnov) == YES, 0.308 p-value (Kolmogrov-Smirnov) == YES!"


Warning message in ks.test(data[["VQ"]], "pnorm", mean = mean(data[["VQ"]]), sd = sd(data[["VQ"]])):
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for Game6 & bitrate of 2000 kbps: 0.48 p-value (Kolmogrov-Smirnov) == YES, 0.039 p-value (Shapiro-Wilk) == NO!"


Warning message in ks.test(data[["VQ"]], "pnorm", mean = mean(data[["VQ"]]), sd = sd(data[["VQ"]])):
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for Game6 & bitrate of 6000 kbps: 0.515 p-value (Kolmogrov-Smirnov) == YES, 0.119 p-value (Kolmogrov-Smirnov) == YES!"


#### Normality is hence likely to not exist within all groups - we are still going to continue though.

### Step 3.3: Homogenity of variances

In [7]:
# Check for homogenity of groups' VQ ratings
get_levene_test_results <- function() {

    test_results <- leveneTest(VQ ~ Game*Bitrate, data = gaming_data %>% mutate(Bitrate = as.character(Bitrate)), center = mean)
    test_results
    n_game1_2000kbps <- (gaming_data %>% dplyr::filter(Game == 'Game1' & Bitrate == 2000) %>% mutate(df = n() - 1))[1,][['df']]
    n_game1_6000kbps <- (gaming_data %>% dplyr::filter(Game == 'Game1' & Bitrate == 6000) %>% mutate(df = n() - 1))[1,][['df']]
    n_game6_2000kbps <- (gaming_data %>% dplyr::filter(Game == 'Game6' & Bitrate == 2000) %>% mutate(df = n() - 1))[1,][['df']]
    n_game6_6000kbps <- (gaming_data %>% dplyr::filter(Game == 'Game6' & Bitrate == 6000) %>% mutate(df = n() - 1))[1,][['df']]

    result <- paste0('F(df_{Game1, 2000 kbps bitrate} = ', n_game1_2000kbps, 
                     ', df_{Game1, 6000 kbps bitrate} = ', n_game1_6000kbps, 
                     ', df_{Game6, 2000 kbps bitrate} = ', n_game6_2000kbps, 
                     ', df_{Game6, 6000 kbps bitrate} = ', n_game6_6000kbps, ') = ',
               round(test_results[1,2], digits=3), 
               ' | p-value = ', 
               round(test_results[1,3], digits=3))
    
    if(test_results[1,3] > 0.05) {
        result <- paste0(result, ' => homogenity of variance CAN be assumed')
    } else {
        result <- paste0(result, ' => homogenity of variance CANNOT be assumed')
    }
    
    result
}

get_levene_test_results()

[1] "F(df_{Game1, 2000 kbps bitrate} = 24, df_{Game1, 6000 kbps bitrate} = 24, df_{Game6, 2000 kbps bitrate} = 21, df_{Game6, 6000 kbps bitrate} = 21) = 1.273 | p-value = 0.289 => homogenity of variance CAN be assumed"

#### Therefore, homogenity of variances can be assumed.

### Step 4: Conduct two-way repeated measure ANOVA

In [8]:
# conduct two-way repeated measure ANOVA
# NOTE here: ezANOVA requires participants to have VQ ratings for all 4 bitrate-game combinations
# => Therefore we need to exclude participants with PID 23, 24, 25 as they only have VQ ratings for 2 conditions

# Also note: "Mauchly's Test for Sphericity: If any within-Ss variables with >2 levels are present, 
# a data frame containing the results of Mauchly's test for Sphericity. Only reported for effects 
# >2 levels because sphericity necessarily holds for effects with only 2 levels" [https://www.rdocumentation.org/packages/ez/versions/3.0-1/topics/ezANOVA]

ezANOVA_data <- gaming_data %>% mutate(Bitrate = as.factor(as.character(Bitrate))) %>% 
                    filter(PID != '23',PID != '24',PID != '25')

anova_res = ezANOVA(
    data = ezANOVA_data, 
    dv = .(VQ), 
    wid = .(PID), 
    within = .(Bitrate, Game)
)

anova_res$ANOVA 

Warning message:
"You have removed one or more Ss from the analysis. Refactoring "PID" for ANOVA."

,Effect,DFn,DFd,F,p,p<.05,ges
2,Bitrate,1,21,2.000348e+02,3.325397e-12,*,3.799115e-01
3,Game,1,21,4.467287e-03,9.473431e-01,,4.972344e-05
4,Bitrate:Game,1,21,6.599115e-02,7.997658e-01,,8.763963e-04


### Step 5: Pairwise comparison / post hoc test

In [9]:
print_bonferroni_results <- function(check_for_bitrate=TRUE) {
    
    if(check_for_bitrate) {
        interpretation <- paste0('Bonferroni p value for effect of bitrate on video quality (VQ) ratings: ')
        p_val <- pairwise.t.test(gaming_data$VQ, gaming_data$Bitrate, p.adj="bonferroni")[['p.value']][1,1]
        interpretation <- paste0(interpretation, p_val)
    } else{
        interpretation <- paste0('Bonferroni p value for effect of game on video quality (VQ) ratings: ')
        p_val <- pairwise.t.test(gaming_data$VQ, gaming_data$Game, p.adj="bonferroni")[['p.value']][1,1]
        interpretation <- paste0(interpretation, p_val)
    }
    
    if(p_val < 0.05) {
        interpretation <- paste0(interpretation, ' => p value < 0.05: significant mean difference')
    } else {
        interpretation <- paste0(interpretation, ' => p value >= 0.05: NO significant mean difference')
    }
    
    interpretation
}

In [10]:
# as almost equal sample sizes and Tukey most widely used: Tukey ["Cramming Sam's tips" for post hoc tests (from lecture)]
# almost equal sample sizes and Bonferroni conservative / widely used: Bonferroni :)
print_bonferroni_results(check_for_bitrate=TRUE)
print_bonferroni_results(check_for_bitrate=FALSE)

# BONUS: almost equal sample sizes and Tukey most widely used: Tukey ["Cramming Sam's tips" for post hoc tests (from lecture)]
TukeyHSD(aov(VQ ~ Bitrate * Game, data = gaming_data))

[1] "Bonferroni p value for effect of bitrate on video quality (VQ) ratings: 5.1687571014263e-11 => p value < 0.05: significant mean difference"

[1] "Bonferroni p value for effect of game on video quality (VQ) ratings: 0.960858305715019 => p value >= 0.05: NO significant mean difference"

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = VQ ~ Bitrate * Game, data = gaming_data)

$Bitrate
              diff      lwr      upr p adj
6000-2000 1.268085 0.925914 1.610256     0

$Game
                   diff        lwr       upr     p adj
Game6-Game1 -0.01063636 -0.3535067 0.3322339 0.9509944

$`Bitrate:Game`
                             diff        lwr        upr     p adj
6000:Game1-2000:Game1  1.31600000  0.6978224  1.9341776 0.0000015
2000:Game6-2000:Game1  0.04054545 -0.5983589  0.6794498 0.9983574
6000:Game6-2000:Game1  1.25418182  0.6152774  1.8930862 0.0000094
2000:Game6-6000:Game1 -1.27545455 -1.9143589 -0.6365502 0.0000066
6000:Game6-6000:Game1 -0.06181818 -0.7007226  0.5770862 0.9942598
6000:Game6-2000:Game6  1.21363636  0.5546568  1.8726159 0.0000339


#### Interaction effect: adjusted p value < 0.05 for (6000 kbps bitrate:Game1)-(2000 kbps bitrate:Game1), (2000 kbps bitrate:Game6)-(6000 kbps bitrate:Game1), (6000 kbps bitrate:Game6)-(2000 kbps bitrate:Game6) 
#### Main effect: adjusted p value < 0.05 for 2000 & 6000 kbps bitrate (changing bitrate), NOT for Game1 & Game6 (switching games)
#### => More: see interpretation

### Step 6: Interpretation

In [11]:
# compute individual degrees of freedom for groups
get_df_of <- function(game, bitrate) {
    paste0('df_{', game, ', ', bitrate, ' kbps bitrate} = ', 
           nrow(gaming_data %>% filter(Game == game & Bitrate == bitrate)) - 1)
}

for (game in games) {
    for (bitrate in bitrates) {
        print(get_df_of(game, bitrate))
    }  
}

[1] "df_{Game1, 2000 kbps bitrate} = 24"
[1] "df_{Game1, 6000 kbps bitrate} = 24"
[1] "df_{Game6, 2000 kbps bitrate} = 21"
[1] "df_{Game6, 6000 kbps bitrate} = 21"


In [12]:
# compute statistics for independent variables' values
gaming_data %>% 
                filter(Bitrate == 2000) %>% 
                group_by(Bitrate) %>% 
                summarize(mean = mean(VQ), sd = sd(VQ))

gaming_data %>% 
                filter(Bitrate == 6000) %>% 
                group_by(Bitrate) %>% 
                summarize(mean = mean(VQ), sd = sd(VQ))

gaming_data %>% 
                filter(Game == 'Game1') %>% 
                group_by(Game) %>% 
                summarize(mean = mean(VQ), sd = sd(VQ))

gaming_data %>% 
                filter(Game == 'Game6') %>% 
                group_by(Game) %>% 
                summarize(mean = mean(VQ), sd = sd(VQ))

Bitrate,mean,sd
2000,2.982979,0.8036112


Bitrate,mean,sd
6000,4.251064,0.8482392


Game,mean,sd
Game1,3.622,1.088022


Game,mean,sd
Game6,3.611364,0.9951548


#### Altering the bitrate or the game does have a significant effect on the video quality (VQ) ratings (alpha significance value = 0.05): 
#### There is no significantly noticable interaction effect between bitrate & game (F statistic value of around 6.6e-02, p-value of around 0.8 with eta² effect size of around 8.76e-04), no significantly noticable main effect of the played game on the video quality (VQ) ratings (F statistic value of around 4.47e-03, p-value of around 0.995 with eta² effect size of around 4.97e-05), BUT a significant rather large main effect of the bitrate on the video quality (VQ) ratings (F statistic value of around 200.00, p-value of around 3.33e-12 with eta² effect size of around 0.38). 
#### The total degrees of freedom are the amount of total observations - 1 => 94 - 1 = 93 [df_{Game1, 2000 kbps bitrate} = 24, df_{Game1, 6000 kbps bitrate} = 24, df_{Game6, 2000 kbps bitrate} = 21, df_{Game6, 6000 kbps bitrate} = 21]. 
#### Regarding pairwise comparisons / post hoc tests, there is a significant statistical difference of ratings between 2000 kbps (mean VQ rating around 2.98, standard deviation around 0.8) & 6000 kbps (mean VQ rating around 4.25, standard deviation around 0.85) bitrate (p-value around 0), but none for the game (p-value around 0.95 > 0.05, where mean VQ rating for Game1 around 3.6 (standard deviation around 1.09), for Game6 around 3.61 (standard deviation around 1.00)) [source: TukeysHSD]. Also, the Bonferroni post hoc test confirms this: bitrate-VQ p value of around 0, game-VQ p value of around 0.96.